# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,7.53,75,100,3.99,GS,1707371638
1,1,blackmans bay,-43.0167,147.3167,21.07,65,3,5.75,AU,1707371644
2,2,punta arenas,-53.1500,-70.9167,9.06,66,0,13.38,CL,1707371654
3,3,wailua homesteads,22.0669,-159.3780,22.44,77,20,8.23,US,1707371630
4,4,port-aux-francais,-49.3500,70.2167,4.40,70,85,11.97,TF,1707371640


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["City", "Humidity"]
)

# Display the map
city_humidity_map


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df

# Drop any rows with null values
clean_city_data_df = city_data_df.dropna()


# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,7.53,75,100,3.99,GS,1707371638
1,1,blackmans bay,-43.0167,147.3167,21.07,65,3,5.75,AU,1707371644
2,2,punta arenas,-53.1500,-70.9167,9.06,66,0,13.38,CL,1707371654
3,3,wailua homesteads,22.0669,-159.3780,22.44,77,20,8.23,US,1707371630
4,4,port-aux-francais,-49.3500,70.2167,4.40,70,85,11.97,TF,1707371640


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country','Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,75,
1,blackmans bay,AU,-43.0167,147.3167,65,
2,punta arenas,CL,-53.1500,-70.9167,66,
3,wailua homesteads,US,22.0669,-159.3780,77,
4,port-aux-francais,TF,-49.3500,70.2167,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10,000
categories = "accommodation.hotel"
limit = 20
params = {
    "categories":categories,
    "apiKey": geoapify_key,
    "limit":limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df['Lat']
    longitude = hotel_df['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"circle:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
grytviken - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
punta arenas - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
labuan - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
keflavik - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
victorica - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
bir el ater - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
bredasdorp - nearest hotel: No hotel found
ciudad lazaro cardenas - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
stanley - nearest hotel: No hotel found
ribeira g

,City,Country,Lat,Lng,Humidity,Hotel Name
0,grytviken,GS,-54.2811,-36.5092,75,No hotel found
1,blackmans bay,AU,-43.0167,147.3167,65,No hotel found
2,punta arenas,CL,-53.1500,-70.9167,66,No hotel found
3,wailua homesteads,US,22.0669,-159.3780,77,No hotel found
4,port-aux-francais,TF,-49.3500,70.2167,70,No hotel found
...,...,...,...,...,...,...
558,kabansk,RU,52.0486,106.6528,70,No hotel found
559,al marj,LY,32.4880,20.8317,60,No hotel found
560,port blair,IN,11.6667,92.7500,42,No hotel found
561,lengshuitan,CN,26.4111,111.5956,90,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    # size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)